<a href="https://colab.research.google.com/github/AIME1995/AIME1995/blob/main/Analyse_du_CACA40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import os
from scipy.stats import norm
import numpy as np
from scipy.stats import norm


# Liste des symboles boursiers des entreprises du CAC 40
tickers = [
    'AC.PA', 'AI.PA', 'BNP.PA', 'CAP.PA', 'CA.PA', 'DG.PA', 'EDR.PA', 'ENGI.PA',
    'OR.PA', 'KER.PA', 'LHN.PA', 'MC.PA', 'ML.PA', 'RNO.PA',
    'SAF.PA', 'SAN.PA', 'SGO.PA', 'SU.PA', 'TOTF.PA', 'VIE.PA'
]

# Dictionnaire des secteurs et sous-secteurs
sector_info = {
    'AC.PA': ['Accor', 'Consumer Services', 'Hotels, Resorts & Cruise Lines'],
    'AI.PA': ['Air Liquide', 'Materials', 'Specialty Chemicals'],
    'BNP.PA': ['BNP Paribas', 'Financials', 'Banks'],
    'CAP.PA': ['Capgemini', 'Information Technology', 'IT Services'],
    'CA.PA': ['Crédit Agricole', 'Financials', 'Banks'],
    'DG.PA': ['Danone', 'Consumer Staples', 'Packaged Foods & Meats'],
    'EDR.PA': ['Eiffage', 'Industrials', 'Construction & Engineering'],
    'ENGI.PA': ['Engie', 'Utilities', 'Utilities'],
    'OR.PA': ['L’Oréal', 'Consumer Staples', 'Personal Products'],
    'KER.PA': ['Kering', 'Consumer Discretionary', 'Textiles, Apparel & Luxury Goods'],
    'LHN.PA': ['LVMH', 'Consumer Discretionary', 'Textiles, Apparel & Luxury Goods'],
    'MC.PA': ['LVMH', 'Consumer Discretionary', 'Textiles, Apparel & Luxury Goods'],
    'ML.PA': ['Michelin', 'Consumer Discretionary', 'Automotive'],
    'RNO.PA': ['Renault', 'Consumer Discretionary', 'Automobiles'],
    'SAF.PA': ['Safran', 'Industrials', 'Aerospace & Defense'],
    'SAN.PA': ['Sanofi', 'Health Care', 'Pharmaceuticals'],
    'SGO.PA': ['Société Générale', 'Financials', 'Banks'],
    'SU.PA': ['Stellantis', 'Consumer Discretionary', 'Automobiles'],
    'TOTF.PA': ['TotalEnergies', 'Energy', 'Oil, Gas & Consumable Fuels'],
    'VIE.PA': ['Vinci', 'Industrials', 'Construction & Engineering']
}

# Fonction pour télécharger les données historiques sur 5 ans
def download_historical_data():
    print("Récupération des données historiques...")
    # Télécharger les données financières sur 5 ans
    data = yf.download(tickers, start="2019-01-01", end=datetime.today().strftime('%Y-%m-%d'), interval="1d", group_by='ticker')

    # Création d'une liste pour stocker les DataFrames de chaque entreprise
    flat_data = []

    # Applatir les données pour chaque ticker
    for ticker in tickers:
        if ticker in data.columns:
            company_data = data[ticker].reset_index()
            company_data['Ticker'] = ticker

            # Ajouter les informations de secteur
            if ticker in sector_info:
                company_data['Company'] = sector_info[ticker][0]
                company_data['Sector'] = sector_info[ticker][1]
                company_data['GICS Sub-Industry'] = sector_info[ticker][2]

            flat_data.append(company_data)

    # Combiner toutes les données financières en un seul DataFrame
    final_data = pd.concat(flat_data, ignore_index=True)

    # Renommer les colonnes pour faciliter l'utilisation
    final_data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Ticker', 'Company', 'Sector', 'GICS Sub-Industry']

    # Sauvegarder les données dans un fichier CSV pour éviter de les télécharger à chaque fois
    final_data.to_csv('historical_data.csv', index=False)
    print("Données historiques sauvegardées dans 'historical_data.csv'.")

# Fonction pour télécharger les données instantanées
def download_instant_data():
    print("Récupération des données instantanées...")
    instant_data = []
    for ticker in tickers:
        stock_info = yf.Ticker(ticker).history(period="1d", interval="1m")  # Données de la journée avec des intervalles d'une minute
        if not stock_info.empty:
            stock_info.reset_index(inplace=True)
            stock_info['Ticker'] = ticker

            # Ajouter les informations de secteur
            if ticker in sector_info:
                stock_info['Company'] = sector_info[ticker][0]
                stock_info['Sector'] = sector_info[ticker][1]
                stock_info['GICS Sub-Industry'] = sector_info[ticker][2]

            instant_data.append(stock_info)

    # Combiner toutes les données instantanées en un seul DataFrame
    instant_df = pd.concat(instant_data, ignore_index=True)
    print(instant_df.tail())  # Afficher les dernières lignes

# Fonction pour récupérer les données de dividendes
def get_dividends_data():
    print("Récupération des données de dividendes...")
    dividends_data = []
    for ticker in tickers:
        stock_info = yf.Ticker(ticker).dividends  # Récupérer l'historique des dividendes
        if not stock_info.empty:
            stock_info = stock_info.reset_index()
            stock_info['Ticker'] = ticker

            # Ajouter les informations de secteur
            if ticker in sector_info:
                stock_info['Company'] = sector_info[ticker][0]
                stock_info['Sector'] = sector_info[ticker][1]
                stock_info['GICS Sub-Industry'] = sector_info[ticker][2]

            dividends_data.append(stock_info)

    # Combiner toutes les données de dividendes en un seul DataFrame
    dividends_df = pd.concat(dividends_data, ignore_index=True)
    print(dividends_df.tail())  # Afficher les dernières lignes
    dividends_df.to_csv('dividends_data.csv', index=False)
    print("Données de dividendes sauvegardées dans 'dividends_data.csv'.")

# Fonction pour récupérer les données de capitalisation boursière
def get_market_cap_data():
    print("Récupération des données de capitalisation boursière...")
    market_cap_data = []
    for ticker in tickers:
        stock_info = yf.Ticker(ticker).info  # Récupérer les informations financières générales
        market_cap = stock_info.get('marketCap', None)  # Capitalisation boursière
        if market_cap is not None:
            market_cap_data.append({
                'Ticker': ticker,
                'Company': sector_info[ticker][0],
                'Market Capitalization': market_cap
            })

    # Créer un DataFrame pour la capitalisation boursière
    market_cap_df = pd.DataFrame(market_cap_data)
    print(market_cap_df.tail())  # Afficher les dernières lignes
    market_cap_df.to_csv('market_cap_data.csv', index=False)
    print("Données de capitalisation boursière sauvegardées dans 'market_cap_data.csv'.")

# Vérifier si les données historiques ont déjà été téléchargées
if not os.path.exists('historical_data.csv'):
    download_historical_data()  # Télécharger et sauvegarder les données historiques si elles n'existent pas encore
else:
    print("Les données historiques sont déjà disponibles dans 'historical_data.csv'.")

# Récupérer les données instantanées, les dividendes et la capitalisation boursière à chaque exécution
download_instant_data()
get_dividends_data()
get_market_cap_data()

# les tables
market_cap_data = pd.read_csv('market_cap_data.csv')
dividends_data = pd.read_csv('dividends_data.csv')
historical_data = pd.read_csv('historical_data.csv')

# Supposons que `historical_data` est le DataFrame contenant les données des actions
# Calculer le logarithme du prix de clôture pour chaque action
historical_data['logreturn'] = np.log(historical_data['Close']).shift(-1) - np.log(historical_data['Close'])

# Créer un nouveau DataFrame pour stocker les résultats par action
result_per_action = historical_data.groupby('Ticker').apply(lambda x: pd.Series({
    'mean_return': x['logreturn'].mean(),                # Moyenne de log_return par action
    'sigma_return': x['logreturn'].std(ddof=1),          # Ecart-type (sigma) de log_return par action
    'mean_return_rolling': x['logreturn'].rolling(window=252).mean().iloc[-1]  # Moyenne mobile sur 252 jours
})).reset_index()

# Calculer la probabilité de chute de 5% pour chaque action et l'ajouter au DataFrame
result_per_action['prob_chute_5%'] = norm.cdf(-0.05, result_per_action['mean_return'], result_per_action['sigma_return'])

# Calculer la probabilité de baisse de 40% sur une année (220 jours) pour chaque action
# On ajuste les paramètres de la distribution pour une période annuelle
annual_mean = result_per_action['mean_return'] * 220
annual_sigma = result_per_action['sigma_return'] * np.sqrt(220)
result_per_action['prob_chute_40%'] = norm.cdf(-0.4, annual_mean, annual_sigma)

# Calculer la Valeur à Risque (VaR) journalière pour chaque action au niveau de confiance de 95 %
result_per_action['VaR_95%'] = norm.ppf(0.05, result_per_action['mean_return'], result_per_action['sigma_return'])

# Ajouter les colonnes des quantiles à 5 % et 95 % pour chaque action
result_per_action['quantile_5%'] = norm.ppf(0.05, result_per_action['mean_return'], result_per_action['sigma_return'])
result_per_action['quantile_95%'] = norm.ppf(0.95, result_per_action['mean_return'], result_per_action['sigma_return'])

# Afficher les résultats
print(result_per_action)


Les données historiques sont déjà disponibles dans 'historical_data.csv'.
Récupération des données instantanées...


ERROR:yfinance:$EDR.PA: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$LHN.PA: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$TOTF.PA: possibly delisted; no price data found  (period=1d)


                      Datetime       Open       High        Low      Close  \
7470 2024-10-11 17:25:00+02:00  29.770000  29.790001  29.770000  29.790001   
7471 2024-10-11 17:26:00+02:00  29.780001  29.780001  29.780001  29.780001   
7472 2024-10-11 17:27:00+02:00  29.780001  29.790001  29.780001  29.790001   
7473 2024-10-11 17:28:00+02:00  29.799999  29.809999  29.799999  29.809999   
7474 2024-10-11 17:29:00+02:00  29.799999  29.850000  29.799999  29.840000   

      Volume  Dividends  Stock Splits  Ticker Company       Sector  \
7470    9111        0.0           0.0  VIE.PA   Vinci  Industrials   
7471    7176        0.0           0.0  VIE.PA   Vinci  Industrials   
7472    5609        0.0           0.0  VIE.PA   Vinci  Industrials   
7473   11877        0.0           0.0  VIE.PA   Vinci  Industrials   
7474   33877        0.0           0.0  VIE.PA   Vinci  Industrials   

               GICS Sub-Industry  
7470  Construction & Engineering  
7471  Construction & Engineering  
7472 

ERROR:yfinance:$EDR.PA: possibly delisted; no timezone found
ERROR:yfinance:$LHN.PA: possibly delisted; no timezone found
ERROR:yfinance:$TOTF.PA: possibly delisted; no price data found  (1d 1925-11-06 -> 2024-10-12)


                         Date  Dividends  Ticker Company       Sector  \
530 2020-05-12 00:00:00+02:00   0.482086  VIE.PA   Vinci  Industrials   
531 2021-05-10 00:00:00+02:00   0.674920  VIE.PA   Vinci  Industrials   
532 2022-07-05 00:00:00+02:00   1.000000  VIE.PA   Vinci  Industrials   
533 2023-05-09 00:00:00+02:00   1.120000  VIE.PA   Vinci  Industrials   
534 2024-05-08 00:00:00+02:00   1.250000  VIE.PA   Vinci  Industrials   

              GICS Sub-Industry  
530  Construction & Engineering  
531  Construction & Engineering  
532  Construction & Engineering  
533  Construction & Engineering  
534  Construction & Engineering  
Données de dividendes sauvegardées dans 'dividends_data.csv'.
Récupération des données de capitalisation boursière...


ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EDR.PA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EDR.PA&crumb=gSlHIq.RmRi


    Ticker           Company  Market Capitalization
12  SAF.PA            Safran            85675827200
13  SAN.PA            Sanofi           125533503488
14  SGO.PA  Société Générale            41193717760
15   SU.PA        Stellantis           135808598016
16  VIE.PA             Vinci            21761509376
Données de capitalisation boursière sauvegardées dans 'market_cap_data.csv'.
     Ticker  mean_return  sigma_return  mean_return_rolling  prob_chute_5%  \
0     AC.PA     0.000525      0.028463             0.003772       0.037940   
1     AI.PA    -0.000480      0.040495            -0.005148       0.110693   
2    BNP.PA     0.000518      0.023136             0.001529       0.014498   
3     CA.PA     0.001066      0.043766             0.005975       0.121646   
4    CAP.PA    -0.001178      0.068060            -0.009584       0.236581   
5     DG.PA     0.000276      0.018520                  NaN       0.003316   
6    EDR.PA          NaN           NaN                  NaN      

<ipython-input-44-0e32ff62bb21>:161: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_per_action = historical_data.groupby('Ticker').apply(lambda x: pd.Series({


In [16]:
market_cap_data = pd.read_csv('market_cap_data.csv')
dividends_data = pd.read_csv('dividends_data.csv')
historical_data = pd.read_csv('historical_data.csv')

In [17]:
market_cap_data.head()


,Ticker,Company,Market Capitalization
0,AC.PA,Accor,9856987136
1,AI.PA,Air Liquide,98504196096
2,BNP.PA,BNP Paribas,70234587136
3,CAP.PA,Capgemini,30799783936
4,CA.PA,Crédit Agricole,9871949824


In [18]:
dividends_data.head()

,Date,Dividends,Ticker,Company,Sector,GICS Sub-Industry
0,2000-06-14 00:00:00+02:00,0.90,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines"
1,2001-06-14 00:00:00+02:00,1.00,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines"
2,2002-06-03 00:00:00+02:00,1.05,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines"
3,2003-06-02 00:00:00+02:00,1.05,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines"
4,2004-01-22 00:00:00+01:00,15.00,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines"


In [37]:
# Cherchons la distribution logarithmique du retour journalier de chaque action.
# Calculer le logarithme du prix de clôture
# historical_data['Log_Return'] = np.log(historical_data['Close'] / historical_data['Close'].shift(1))
historical_data["logreturn"] = np.log(historical_data['Close']).shift(-1) - np.log(historical_data['Close'])
# historical_data['logreturnB'] = historical_data.groupby('Ticker')['Close'].shift(-1) - np.log(historical_data['Close'])

# Calcumons la moyenne mobile du retour journalier
historical_data["mean_return"] = historical_data["logreturn"].rolling(window=252).mean()
# Calculons la moyenne de log_return
moyenne_Log_Return = historical_data['logreturn'].mean()
# Calculons le stigma du log_return
historical_data["sigma_return"] = historical_data['logreturn'].std(ddof=1)

#Claculer la probabilité de chutte des différentes actions de 5%
prob_return_baisse_5 = norm.cdf(-0.05, historical_data['mean_return'], historical_data['sigma_return'])

historical_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Company,Sector,GICS Sub-Industry,Log_Return,logreturn,mean_return,sigma_return,prob_fall,logreturnB
0,2019-01-02 00:00:00+00:00,36.939999,36.939999,36.040001,36.599998,33.734222,562162.0,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines",NaN,-0.016529,NaN,0.041189,NaN,32.399952
1,2019-01-03 00:00:00+00:00,36.389999,36.549999,36.000000,36.000000,33.181202,800687.0,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines",-0.016529,0.023065,NaN,0.041189,NaN,33.256481
2,2019-01-04 00:00:00+00:00,36.410000,37.029999,36.169998,36.840000,33.955429,1074737.0,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines",0.023065,0.004334,NaN,0.041189,NaN,33.393416
3,2019-01-07 00:00:00+00:00,37.020000,37.020000,36.439999,37.000000,34.102901,554260.0,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines",0.004334,0.002429,NaN,0.041189,NaN,33.479082
4,2019-01-08 00:00:00+00:00,37.099998,37.459999,36.869999,37.090000,34.185856,720341.0,AC.PA,Accor,Consumer Services,"Hotels, Resorts & Cruise Lines",0.002429,0.012060,NaN,0.041189,NaN,33.926654


In [26]:
moyenne_action = historical_data.groupby('Ticker')['logreturn'].mean()
sigma_action = historical_data.groupby('Ticker')['logreturn'].std(ddof=1)

In [38]:
import scipy.stats as stats
import pandas as pd

# Calcul de la moyenne des rendements par action
mean_per_action = historical_data.groupby('Ticker')['logreturn'].mean().reset_index()
mean_per_action = mean_per_action.rename(columns={'logreturn': 'mean_return'})

# Calcul de l'écart-type (sigma) des rendements par action
sigma_per_action = historical_data.groupby('Ticker')['logreturn'].std(ddof=1).reset_index()
sigma_per_action = sigma_per_action.rename(columns={'logreturn': 'sigma'})


# Fusionner les moyennes et les écarts-types dans un seul DataFrame
result = pd.merge(mean_per_action, sigma_per_action, on='Ticker')

# Calcul de la probabilité de chute de 5% pour chaque action
result['prob_chute_5%'] = result.apply(
    lambda row: stats.norm.cdf(-0.05, loc=row['mean_return'], scale=row['sigma']), axis=1
)

#Probabilité de baisse de 40% en une année (220jours)

# Afficher les résultats
print(result)


     Ticker  mean_return     sigma  prob_chute_5%
0     AC.PA     0.000525  0.028463       0.037940
1     AI.PA    -0.000480  0.040495       0.110693
2    BNP.PA     0.000518  0.023136       0.014498
3     CA.PA     0.001066  0.043766       0.121646
4    CAP.PA    -0.001178  0.068060       0.236581
5     DG.PA     0.000276  0.018520       0.003316
6    EDR.PA          NaN       NaN            NaN
7   ENGI.PA     0.001865  0.067845       0.222296
8    KER.PA    -0.000352  0.020441       0.007573
9    LHN.PA          NaN       NaN            NaN
10    MC.PA    -0.001665  0.090620       0.296886
11    ML.PA     0.000610  0.021031       0.008054
12    OR.PA     0.000473  0.015011       0.000386
13   RNO.PA     0.000452  0.038028       0.092304
14   SAF.PA    -0.000217  0.035393       0.079778
15   SAN.PA    -0.000645  0.035440       0.081869
16   SGO.PA     0.000486  0.022049       0.011018
17    SU.PA     0.000948  0.017586       0.001884
18  TOTF.PA          NaN       NaN            NaN


In [42]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Supposons que `historical_data` est le DataFrame contenant les données des actions
# Calculer le logarithme du prix de clôture pour chaque action
historical_data['logreturn'] = np.log(historical_data['Close']).shift(-1) - np.log(historical_data['Close'])

# Créer un nouveau DataFrame pour stocker les résultats par action
result_per_action = historical_data.groupby('Ticker').apply(lambda x: pd.Series({
    'mean_return': x['logreturn'].mean(),                # Moyenne de log_return par action
    'sigma_return': x['logreturn'].std(ddof=1),          # Ecart-type (sigma) de log_return par action
    'mean_return_rolling': x['logreturn'].rolling(window=252).mean().iloc[-1]  # Moyenne mobile sur 252 jours
})).reset_index()

# Calculer la probabilité de chute de 5% pour chaque action et l'ajouter au DataFrame
result_per_action['prob_chute_5%'] = norm.cdf(-0.05, result_per_action['mean_return'], result_per_action['sigma_return'])

# Calculer la probabilité de baisse de 40% sur une année (220 jours) pour chaque action
# On ajuste les paramètres de la distribution pour une période annuelle
annual_mean = result_per_action['mean_return'] * 220
annual_sigma = result_per_action['sigma_return'] * np.sqrt(220)
result_per_action['prob_chute_40%'] = norm.cdf(-0.4, annual_mean, annual_sigma)

# Calculer la Valeur à Risque (VaR) journalière pour chaque action au niveau de confiance de 95 %
result_per_action['VaR_95%'] = norm.ppf(0.05, result_per_action['mean_return'], result_per_action['sigma_return'])

# Ajouter les colonnes des quantiles à 5 % et 95 % pour chaque action
result_per_action['quantile_5%'] = norm.ppf(0.05, result_per_action['mean_return'], result_per_action['sigma_return'])
result_per_action['quantile_95%'] = norm.ppf(0.95, result_per_action['mean_return'], result_per_action['sigma_return'])

# Afficher les résultats
print(result_per_action)


     Ticker  mean_return  sigma_return  mean_return_rolling  prob_chute_5%  \
0     AC.PA     0.000525      0.028463             0.003772       0.037940   
1     AI.PA    -0.000480      0.040495            -0.005148       0.110693   
2    BNP.PA     0.000518      0.023136             0.001529       0.014498   
3     CA.PA     0.001066      0.043766             0.005975       0.121646   
4    CAP.PA    -0.001178      0.068060            -0.009584       0.236581   
5     DG.PA     0.000276      0.018520                  NaN       0.003316   
6    EDR.PA          NaN           NaN                  NaN            NaN   
7   ENGI.PA     0.001865      0.067845             0.010346       0.222296   
8    KER.PA    -0.000352      0.020441                  NaN       0.007573   
9    LHN.PA          NaN           NaN                  NaN            NaN   
10    MC.PA    -0.001665      0.090620            -0.013653       0.296886   
11    ML.PA     0.000610      0.021031             0.002419     

<ipython-input-42-315762553cf1>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_per_action = historical_data.groupby('Ticker').apply(lambda x: pd.Series({


In [49]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import os
from scipy.stats import norm
import numpy as np

# Liste des symboles boursiers des entreprises du CAC 40
tickers = [
    'AC.PA', 'AI.PA', 'BNP.PA', 'CAP.PA', 'CA.PA', 'DG.PA', 'EDR.PA', 'ENGI.PA',
    'OR.PA', 'KER.PA', 'LHN.PA', 'MC.PA', 'ML.PA', 'RNO.PA',
    'SAF.PA', 'SAN.PA', 'SGO.PA', 'SU.PA', 'TOTF.PA', 'VIE.PA'
]

# Dictionnaire des secteurs et sous-secteurs
sector_info = {
    'AC.PA': ['Accor', 'Consumer Services', 'Hotels, Resorts & Cruise Lines'],
    'AI.PA': ['Air Liquide', 'Materials', 'Specialty Chemicals'],
    'BNP.PA': ['BNP Paribas', 'Financials', 'Banks'],
    'CAP.PA': ['Capgemini', 'Information Technology', 'IT Services'],
    'CA.PA': ['Crédit Agricole', 'Financials', 'Banks'],
    'DG.PA': ['Danone', 'Consumer Staples', 'Packaged Foods & Meats'],
    'EDR.PA': ['Eiffage', 'Industrials', 'Construction & Engineering'],
    'ENGI.PA': ['Engie', 'Utilities', 'Utilities'],
    'OR.PA': ['L’Oréal', 'Consumer Staples', 'Personal Products'],
    'KER.PA': ['Kering', 'Consumer Discretionary', 'Textiles, Apparel & Luxury Goods'],
    'LHN.PA': ['LVMH', 'Consumer Discretionary', 'Textiles, Apparel & Luxury Goods'],
    'MC.PA': ['LVMH', 'Consumer Discretionary', 'Textiles, Apparel & Luxury Goods'],
    'ML.PA': ['Michelin', 'Consumer Discretionary', 'Automotive'],
    'RNO.PA': ['Renault', 'Consumer Discretionary', 'Automobiles'],
    'SAF.PA': ['Safran', 'Industrials', 'Aerospace & Defense'],
    'SAN.PA': ['Sanofi', 'Health Care', 'Pharmaceuticals'],
    'SGO.PA': ['Société Générale', 'Financials', 'Banks'],
    'SU.PA': ['Stellantis', 'Consumer Discretionary', 'Automobiles'],
    'TOTF.PA': ['TotalEnergies', 'Energy', 'Oil, Gas & Consumable Fuels'],
    'VIE.PA': ['Vinci', 'Industrials', 'Construction & Engineering']
}

# Fonction pour télécharger les données historiques sur 5 ans
def download_historical_data():
    print("Récupération des données historiques...")
    data = yf.download(tickers, start="2019-01-01", end=datetime.today().strftime('%Y-%m-%d'), interval="1d", group_by='ticker')
    flat_data = []
    for ticker in tickers:
        if ticker in data.columns:
            company_data = data[ticker].reset_index()
            company_data['Ticker'] = ticker

            if ticker in sector_info:
                company_data['Company'] = sector_info[ticker][0]
                company_data['Sector'] = sector_info[ticker][1]
                company_data['GICS Sub-Industry'] = sector_info[ticker][2]

            flat_data.append(company_data)

    final_data = pd.concat(flat_data, ignore_index=True)
    final_data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Ticker', 'Company', 'Sector', 'GICS Sub-Industry']
    final_data.to_csv('historical_data.csv', index=False)
    print("Données historiques sauvegardées dans 'historical_data.csv'.")

# Vérifier si les données historiques ont déjà été téléchargées
if not os.path.exists('historical_data.csv'):
    download_historical_data()

# Charger les données historiques
historical_data = pd.read_csv('historical_data.csv')

# Calculer les logreturns
historical_data['logreturn'] = np.log(historical_data['Close']).shift(-1) - np.log(historical_data['Close'])

# Créer un nouveau DataFrame pour stocker les résultats par action
result_per_action = historical_data.groupby('Ticker').apply(lambda x: pd.Series({
    'mean_return': x['logreturn'].mean(),                # Moyenne de log_return par action
    'sigma_return': x['logreturn'].std(ddof=1),          # Ecart-type (sigma) de log_return par action
    'mean_return_rolling': x['logreturn'].rolling(window=252).mean().iloc[-1],  # Moyenne mobile sur 252 jours
    'volatility_annual': x['logreturn'].std(ddof=1) * np.sqrt(252),  # Volatilité annuelle
    'volatility_30d': x['logreturn'].rolling(window=30).std(ddof=1).iloc[-1],  # Volatilité à 30 jours
})).reset_index()

# Calculer le rendement en dividende (supposons une valeur fixe pour cet exemple)
result_per_action['dividend_yield'] = [0.02] * len(result_per_action)  # Exemple avec un rendement de 2%

# Calculer l'alpha et le bêta par rapport à un indice de référence (par exemple, CAC 40, supposons une moyenne fictive)
market_return = 0.07  # Rendement moyen du marché (exemple fictif)
risk_free_rate = 0.01  # Taux sans risque
result_per_action['beta'] = (result_per_action['mean_return'] - risk_free_rate) / (market_return - risk_free_rate)  # Calcul de bêta
result_per_action['alpha'] = result_per_action['mean_return'] - result_per_action['beta'] * market_return  # Calcul d'alpha

# Ajouter des mesures de performance
result_per_action['prob_chute_5%'] = norm.cdf(-0.05, result_per_action['mean_return'], result_per_action['sigma_return'])
annual_mean = result_per_action['mean_return'] * 220
annual_sigma = result_per_action['sigma_return'] * np.sqrt(220)
result_per_action['prob_chute_40%'] = norm.cdf(-0.4, annual_mean, annual_sigma)
result_per_action['VaR_95%'] = norm.ppf(0.05, result_per_action['mean_return'], result_per_action['sigma_return'])
result_per_action['quantile_5%'] = norm.ppf(0.05, result_per_action['mean_return'], result_per_action['sigma_return'])
result_per_action['quantile_95%'] = norm.ppf(0.95, result_per_action['mean_return'], result_per_action['sigma_return'])

# Ajouter les informations de secteur
for ticker in tickers:
    if ticker in sector_info:
        result_per_action.loc[result_per_action['Ticker'] == ticker, 'Company'] = sector_info[ticker][0]
        result_per_action.loc[result_per_action['Ticker'] == ticker, 'Sector'] = sector_info[ticker][1]
        result_per_action.loc[result_per_action['Ticker'] == ticker, 'GICS Sub-Industry'] = sector_info[ticker][2]

# Afficher les résultats
print(result_per_action)
historical_data.tail()

     Ticker  mean_return  sigma_return  mean_return_rolling  \
0     AC.PA     0.000525      0.028463             0.003772   
1     AI.PA    -0.000480      0.040495            -0.005148   
2    BNP.PA     0.000518      0.023136             0.001529   
3     CA.PA     0.001066      0.043766             0.005975   
4    CAP.PA    -0.001178      0.068060            -0.009584   
5     DG.PA     0.000276      0.018520                  NaN   
6    EDR.PA          NaN           NaN                  NaN   
7   ENGI.PA     0.001865      0.067845             0.010346   
8    KER.PA    -0.000352      0.020441                  NaN   
9    LHN.PA          NaN           NaN                  NaN   
10    MC.PA    -0.001665      0.090620            -0.013653   
11    ML.PA     0.000610      0.021031             0.002419   
12    OR.PA     0.000473      0.015011             0.000199   
13   RNO.PA     0.000452      0.038028             0.004181   
14   SAF.PA    -0.000217      0.035393            -0.00

<ipython-input-49-27128d41fea8>:72: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_per_action = historical_data.groupby('Ticker').apply(lambda x: pd.Series({


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Company,Sector,GICS Sub-Industry,logreturn
29675,2024-10-07 00:00:00+00:00,29.510000,29.530001,29.100000,29.170000,29.170000,1166253.0,VIE.PA,Vinci,Industrials,Construction & Engineering,0.003764
29676,2024-10-08 00:00:00+00:00,29.000000,29.280001,28.830000,29.280001,29.280001,1162477.0,VIE.PA,Vinci,Industrials,Construction & Engineering,-0.001025
29677,2024-10-09 00:00:00+00:00,29.299999,29.440001,29.219999,29.250000,29.250000,1140499.0,VIE.PA,Vinci,Industrials,Construction & Engineering,0.008172
29678,2024-10-10 00:00:00+00:00,29.270000,29.629999,29.270000,29.490000,29.490000,1060814.0,VIE.PA,Vinci,Industrials,Construction & Engineering,0.010457
29679,2024-10-11 00:00:00+00:00,29.559999,29.860001,29.530001,29.799999,29.799999,1335323.0,VIE.PA,Vinci,Industrials,Construction & Engineering,NaN
